# Understanding dropouts

## Introduction
Le **surapprentissage** est un problème majeur dans l'entraînement des réseaux de neurones profonds. Une technique efficace pour le combattre est le **Dropout**.

Dans ce notebook, nous allons :
1. Mais le dropout, c'est quoi ?
2. Exemple d'application de la technique du Dropout.
3. Comparer les performances d'un réseau avec et sans Dropout.
4. Généralisation du dropout

***
## 1. Mais le dropout, c'est quoi ?

### Un peu d'histoire
Avant l’introduction du Dropout, plusieurs approches avaient déjà exploré l’idée d’ajouter du bruit aux réseaux de neurones pour améliorer leur généralisation. **Hanson (1990)** a proposé la **Stochastic Delta Rule**, qui injectait du bruit dans l’apprentissage des poids pour limiter le surapprentissage. **Bishop (1995)** a démontré que **perturber les entrées** d’un modèle pouvait être interprété comme une forme de régularisation bayésienne. **LeCun et al. (1998)** ont testé l’**ajout de bruit** dans les activations des neurones pour limiter la dépendance excessive aux données d'entraînement. D’autres travaux, comme ceux de **Hinton & Nowlan (1992)** et **Neal (1995, 2001)**, ont étudié l’utilisation de **distributions probabilistes sur les poids et les activations** afin d’améliorer la généralisation des modèles.

Le **Dropout** a été introduit en **2014** par **Srivastava et al.** dans leur article **"Dropout: A Simple Way to Prevent Neural Networks from Overfitting"**. Cette technique a été développée pour pallier le problème du surapprentissage dans les réseaux de neurones profonds. Avant son introduction, les méthodes classiques de régularisation comme la **pénalisation L2** et le **early stopping** étaient couramment utilisées, mais elles ne suffisaient pas toujours à éviter l'adaptation excessive aux données d'entraînement.

L'idée principale derrière le Dropout était inspirée de l'**apprentissage par ensembles**, où plusieurs modèles indépendants sont combinés pour améliorer la généralisation. Toutefois, entraîner et stocker plusieurs réseaux de neurones profonds était **coûteux en calcul**. Srivastava et son équipe ont alors cherché un moyen d'obtenir un effet similaire au **model averaging**, mais de manière bien **plus efficace**.


### Fonctionnement du Dropout

Leur solution a été de **perturber l'apprentissage en désactivant aléatoirement des neurones à chaque itération**, forçant ainsi chaque neurone à apprendre des représentations **plus robustes** sans dépendre excessivement de neurones spécifiques. En empêchant la formation de **co-adaptations trop spécialisées**, cette approche a conduit à des modèles généralisant mieux aux données non vues. De plus, lors de la phase de **test**, tous les **neurones sont activés** mais leurs **poids sont ajustés** pour compenser les désactivations précédentes, simulant ainsi un moyennage implicite d'un grand nombre de **réseaux plus petits entraînés en parallèle**.


### Impact sur l’apprentissage des réseaux de neurones
Le Dropout a plusieurs effets bénéfiques sur l'apprentissage :
- **Réduction du surapprentissage** : en empêchant les neurones de trop s’adapter aux données d’entraînement.
- **Amélioration de la robustesse** : chaque neurone doit apprendre des représentations plus générales, car il ne peut pas compter sur d’autres neurones spécifiques.
- **Effet d’ensemble (ensemble learning)** : en échantillonnant différents sous-réseaux à chaque itération, le modèle final se comporte comme une combinaison de plusieurs réseaux différents, ce qui améliore la généralisation.


***
## 2. Exemple d'application de la technique du Dropout.

### Présentation du modèle 

Ca sera plus simple pour tout le monde si je reprends un modèle que l'on connait. On va tout simplement utiliser le modèle que nous avions défini au TD de deep learning sur le jeu de données FASHION-MNIST. Nous partirons de là et nous appliquerons la technique du dropout pour vraiment comprendre l'interêt de ce procédé.

[Fashion-MNIST](https://github.com/zalandoresearch/fashion-mnist) est un jeu de données contenant des images d'articles de Zalando, composé d'un ensemble d'entraînement de 60 000 exemples et d'un ensemble de test de 10 000 exemples. Chaque exemple est une image en niveaux de gris de 28x28 pixels, associée à une étiquette parmi 10 classes. 

<img src="img/fashion-mnist-small.png">

In [ ]:
# Importation des librairies
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

# Fonction d'entraînement avec suivi de l'historique
def train(model, trainloader, validloader, epochs=5, lr=0.001):
    optimizer = torch.optim.Adam(model.parameters(), lr)
    criterion = nn.CrossEntropyLoss() # La perte est définie par la Cross Entropy Loss
    train_history, valid_history = [], []

    for epoch in range(epochs):
        model.train()  # Mode entraînement (dropout activé, si dropout il y a)
        running_loss = 0.0
        correct_train = 0
        total_train = 0

        for images, labels in trainloader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            # Suivi de l'exactitude de l'entraînement
            _, predicted = torch.max(outputs, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        # Suivi de la perte et de l'exactitude de l'entraînement
        train_loss = running_loss / len(trainloader)
        train_acc = 100 * correct_train / total_train
        train_history.append((train_loss, train_acc))

        # Validation après chaque époque
        model.eval()  # Mode évaluation (dropout désactivé, si dropout il y a)
        valid_loss = 0.0
        correct_valid = 0
        total_valid = 0
        with torch.no_grad():
            for images, labels in validloader:
                outputs = model(images)
                loss = criterion(outputs, labels)
                valid_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total_valid += labels.size(0)
                correct_valid += (predicted == labels).sum().item()

        # Suivi de la perte et de l'exactitude de la validation
        valid_loss = valid_loss / len(validloader)
        valid_acc = 100 * correct_valid / total_valid
        valid_history.append((valid_loss, valid_acc))

        print(f"Epoch {epoch+1} - Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.2f}%")
        print(f"Epoch {epoch+1} - Valid Loss: {valid_loss:.4f}, Valid Accuracy: {valid_acc:.2f}%")

    return train_history, valid_history

# Visualisation des courbes de perte et de précision
def plot_train_val(train_history, valid_history):
    train_loss, train_acc = zip(*train_history)
    valid_loss, valid_acc = zip(*valid_history)

    # Tracer la courbe de perte
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(train_loss, label='Train Loss')
    plt.plot(valid_loss, label='Valid Loss')
    plt.title('Loss per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Tracer la courbe de précision
    plt.subplot(1, 2, 2)
    plt.plot(train_acc, label='Train Accuracy')
    plt.plot(valid_acc, label='Valid Accuracy')
    plt.title('Accuracy per Epoch')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
# Charger le dataset Fashion-MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

dataset = torchvision.datasets.FashionMNIST(root="data", train=True, download=True, transform=transform)
trainset, validset = random_split(dataset, (50000, 10000))
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
validloader = DataLoader(validset, batch_size=64, shuffle=False)

testset = torchvision.datasets.FashionMNIST(root='data', train=False, download=True, transform=transform)

Ici, nous choisissons de définir un batch de taille 64. Nous utilisons une base de données d'entrainement de 50000 images et on validera le modèle sur 10000 images.

Nous allons ensuite définir la classe de réseaux de neurones denses (sans dropout).

In [1]:
# Définition du modèle sans Dropout
class ReLUNet(nn.Module):
    def __init__(self):
        super(ReLUNet, self).__init__()
        self.fc1 = nn.Linear(784, 120)
        self.fc2 = nn.Linear(120,60)
        self.fc3 = nn.Linear(60, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

NameError: name 'nn' is not defined

La classe est définie, nous allons maintenant instancier le modèle, le tester et vérifier son erreur. Notre fonction d'entrainement/test mémorise les scores qu'on va pouvoir comparer avec ceux du dropout plus loin dans le notebook.

In [ ]:
# instanciation -> entraînement -> test du modèle sans dropout
model = ReLUNet()
net_no_dropout = ReLUNet()  # Créer le modèle sans dropout
train_history_no_dropout, valid_history_no_dropout = train(
    net_no_dropout, 
    trainloader, validloader, 
    epochs=5, 
    lr=0.001
    )
plot_train_val(train_history_no_dropout, valid_history_no_dropout)

Pour pouvoir nettement observer ces résultats il faudrait aller plus loins dans les epochs mais par soucis de temps je vous mets une capture des résultats obtenus avec 50 epochs : 

<figure>
    <img src="img/No Dropout - 50 epochs.png" style="width: 900px; height: auto;">
    <figcaption><strong>Figure 1 :</strong> Résultats durant 50 epochs sans dropout (surapprentissage)</figcaption>
</figure>

### Conclusion

Sans le dropout, le modèle a tendance à surapprendre, nous voyons alors que lors de la validation, l'erreur diverge.
***
### Application du dropout

Nous allons ici étapes par étapes appliquer le dropout à notre modèle. Les différentes étapes seront présentées sous forme de questions auquelles vous pourrez tenter de répondre. Il est fortement conseillé de tenter de résoudre les questions par vous même  pour comprendre le dropout en pratique et savoir l'appliquer.

Torch utilise le dropout spatial, qui est décrit ici : https://arxiv.org/pdf/1411.4280.pdf

<div class="alert alert-success">
Question 1 : Complète le modèle suivant avec un dropout sur la première couche cachée. Pour commencer, tu peux poser p=0.5 (50% de chance d'être désactivée).
</div>

In [ ]:
# %load solutions/Q1.py
class ModelWithSingleDropout(nn.Module):
    def __init__(self):
        super(ModelWithSingleDropout, self).__init__()
        self.fc1 = nn.Linear(784, 120)
        
        ... # Créer le dropout sur cette couche à l'aide de la fonction nn.Dropout(p)
        
        self.fc2 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = F.relu(self.fc1(x))
        
        ... # Appliquer le dropout après la première couche cachée
        
        x = self.fc2(x)
        return x

Ici, vous avez ajouté une couche Dropout après la première couche cachée. Cela permet de "désactiver" aléatoirement une partie des neurones de cette couche pendant l'entraînement. Vous voyez que le dropout est facile à implémenter dans un modèle. Par la suite vous allez implémenter le dropout sur chaque couche de notre modèle de classification (sauf la couche finale) et visualiser son effet et apprendre à optimiser ses paramètres.

<div class="alert alert-success">
Question 2 : Maintenant, modifie le modèle de classification ReLUNet en ajoutant plusieurs couches de dropout dans le réseau. Définis les dropouts comme suit : la première couche de dropout doit avoir p=0.25 et la deuxième et troisième soivent avoir p=0.4.
</div>

In [ ]:
# %load solutions/Q2.py
class RandomDropoutModel(nn.Module):
    def __init__(self):
        ...
        # A compléter

    def forward(self, x):
        ...
        # A compléter

In [ ]:
# Instanciation -> entraînement -> test du modèle avec dropouts
net = RandomDropoutModel()
train_history_with_dropout, valid_history_with_dropout = train(
    net, 
    trainloader, validloader, 
    epochs=5,
    lr=0.001,
    )
plot_train_val(train_history_with_dropout, valid_history_with_dropout)

Vous vous demandez pourquoi on utilise ces valeurs précises ? Dans le prochain code vous allez essayer avec d'autres valeurs, vous comprendrez. Essaye avec des petites et grandes valeurs pour bien visualiser l'effet du dropout.

<div class="alert alert-success">
Question 3 : Testez d'autres valeurs. Essayez avec des petites et grandes valeurs pour bien visualiser le comportement du modèle.
</div>

In [ ]:
# Définition de ton modèle avec Dropout
class YourDropoutModel(nn.Module):
    def __init__(self):
        super(YourDropoutModel, self).__init__()
        self.dropout1 = nn.Dropout(p=...)
        self.fc1 = nn.Linear(784, 120)
        self.dropout2 = nn.Dropout(p=...)
        self.fc2 = nn.Linear(120, 60)
        self.dropout3 = nn.Dropout(p=...)
        self.fc3 = nn.Linear(60, 10)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.dropout1(x)  # Appliquer le premier dropout
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)  # Appliquer le deuxième dropout
        x = F.relu(self.fc2(x))
        x = self.dropout3(x)  # Appliquer le troisième dropout
        x = self.fc3(x)
        return x

In [ ]:
# Instanciation et entraînement de ton modèle avec dropouts aléatoires
yournet = YourDropoutModel()
train_history_your_dropout, valid_history_your_dropout = train(
    yournet, 
    trainloader, validloader, 
    epochs=5,
    lr=..., # Ici tu peux le modifier pour visualiser son effet, 
# il est conseillé de l'augmenter légèrement par rapport au modèle sans dropout pour contrer le bruit généré
    earlystopping=False)
plot_train_val(train_history_your_dropout, valid_history_your_dropout)

Pour pouvoir nettement observer ces résultats il faudrait aller plus loins dans les epochs mais par soucis de temps je vous mets une capture des résultats obtenus avec 50 epochs : 

<figure>
    <img src="img/several Dropout's config.png" style="width: 900px; height: auto;">
    <figcaption><strong>Figure 2 :</strong> Résultats durant 50 epochs avec différentes configurations du dropout en fonction de leur pourcentage moyen de désactivation</figcaption>
</figure>

Vous comprennez maintenant l'interêt de fixer des pourcentages précis, dans notre cas le résultat optimal est obtenu avec p1=0.25, p2=0.4 et p3=0.4. Dans la plupart des modèles on à une disposition optimale des probabilités de désactivation comme suit :
- 25% de dropout dans les premières couches : garde une bonne quantité d'informations pour apprendre des représentations générales des données tout en réduisant le risque de surapprentissage.
- 40% de dropout dans les couches profondes : favorise la généralisation en forçant le modèle à apprendre des représentations plus robustes et moins spécifiques aux données d'entraînement.

Ainsi, la stratégie consiste à augmenter le dropout dans les couches profondes, où le modèle est déjà plus spécialisé, et à le garder modéré dans les couches initiales pour ne pas perdre des informations essentielles trop tôt.

### Bravo ! 
Vous avez appliqué le dropout au modèle de reconnaissance de chiffres de MNIST. J'espère que ca s'est bien passé et que vous avez bien saisi comment on implémente du dropout dans les différentes couches d'un réseau de neurones et comment on l'optimise. Nous verrons ensuite son effet sur les résultats du modèle. 

Je me permets de te transmettre un petit message du créateur de ce notebook :

*Salut à toi jeune codeur ! J'espère que ce petit message va te faire sourire au milieu de tes corrections de notebooks interminables. Je te souhaite bon courage pour la suite et surtout n'oublie pas de me mettre une bonne note, sinon je serais pas content. Bisous*
***

## 3. Comparer les performances d'un réseau avec et sans Dropout.

Le code suivant permet de comparer les performances du modèles avec dropout "optimisé" et du modèle sans dropout.


In [ ]:
# Créer la figure avec les 2 résultats précédents
fig, axs = plt.subplots(1, 2, figsize=(14, 6))

# Tracer la courbe de perte (Loss)
axs[0].plot([x[0] for x in train_history_no_dropout], label='Train Loss (No Dropout)', color='blue')
axs[0].plot([x[0] for x in valid_history_no_dropout], label='Valid Loss (No Dropout)', color='red')
axs[0].plot([x[0] for x in train_history_with_dropout], label='Train Loss (With Dropout)', color='green')
axs[0].plot([x[0] for x in valid_history_with_dropout], label='Valid Loss (With Dropout)', color='orange')
axs[0].plot([x[0] for x in train_history_your_dropout], label='Train Loss (Your Model)', color='yellow')
axs[0].plot([x[0] for x in valid_history_your_dropout], label='Valid Loss (Your Model)', color='pink')
axs[0].set_title('Perte (Loss) par époque')
axs[0].set_xlabel('Époques')
axs[0].set_ylabel('Perte')
axs[0].legend()

# Tracer la courbe de précision (Accuracy)
axs[1].plot([x[1] for x in train_history_no_dropout], label='Train Accuracy (No Dropout)', color='blue')
axs[1].plot([x[1] for x in valid_history_no_dropout], label='Valid Accuracy (No Dropout)', color='red')
axs[1].plot([x[1] for x in train_history_with_dropout], label='Train Accuracy (With Dropout)', color='green')
axs[1].plot([x[1] for x in valid_history_with_dropout], label='Valid Accuracy (With Dropout)', color='orange')
axs[1].plot([x[1] for x in train_history_your_dropout], label='Train Accuracy (Your Model)', color='yellow')
axs[1].plot([x[1] for x in valid_history_your_dropout], label='Valid Accuracy (Your Model)', color='pink')
axs[1].set_title('Précision (Accuracy) par époque')
axs[1].set_xlabel('Époques')
axs[1].set_ylabel('Précision (%)')
axs[1].legend()

# Afficher le graphique
plt.tight_layout()
plt.show()

Pour pouvoir nettement observer ces résultats il faudrait aller plus loin dans les epochs mais par soucis de temps je vous mets une capture des résultats obtenus avec 50 et 200 epochs : 

<figure>
    <img src="img/resultats - p1=0,25 - p2=0,4 - p3=0,4 - 50 epochs.png" style="width: 900px; height: auto;">
    <figcaption><strong>Figure 3 :</strong> Résultats durant 50 epochs avec p1=0.25, p2=0.4 et p3=0.4.</figcaption>
</figure>
<figure>
    <img src="img/resultats - p1=0,25 - p2=0,4 - p3=0,4 - 200 epochs.png" style="width: 900px; height: auto;">
    <figcaption><strong>Figure 4 :</strong> Résultats durant 200 epochs avec p1=0.25, p2=0.4 et p3=0.4.</figcaption>
</figure>


Dans cet exemple, on peut voir que l'utilisation de dropout permet de bien limiter les effets du surapprentissage mais il vient aussi agir sur la vitesse de convergence du modèle. C'est donc pour cela qu'il faut choisir méticuleusement les probabilités de dropout pour limiter le bruit qui vient altérer l'efficacité du modèle. Ici je pense que le modèle aurait pu encore être optimiser pour obtenir une meilleure convergence de la précision en utilisant des probabilités légèrement plus précises.

Maintenant que vous avez vu en pratique comment s'applique et s'optimise le dropout sur un modèle de réseaux de neurones denses. Je vais vous présenter comment le dropout peut s'appliquer à d'autres architectures de réseaux et comment on peut le combiner à d'autres techniques pour l'optimiser.

***
## 4. Généralisation du dropout

### 4.1 Extension du Dropout à d'autres modèles

Dans la section précédente, nous avons appliqué le **dropout** à un modèle de classification simple sur le dataset **Fashion-MNIST**. Cependant, cette technique de régularisation ne se limite pas aux **réseaux de neurones denses**. 

Le dropout se généralise efficacement à d’autres architectures. Dans les réseaux de neurones convolutifs (*CNNs*), il est particulièrement utile dans les dernières couches pleinement connectées. Les CNNs sont conçus pour extraire des caractéristiques spatiales à travers des filtres convolutifs, et le dropout permet d’empêcher ces filtres de surapprendre des motifs spécifiques à l’ensemble d’entraînement. En désactivant aléatoirement des neurones dans les couches finales, le réseau est contraint d’apprendre des représentations plus générales, ce qui améliore la capacité de généralisation sur de nouvelles images.

Dans les réseaux récurrents (*RNNs*), le dropout est adapté sous la forme du *Variational Dropout*. Contrairement aux réseaux feedforward, les RNNs conservent une mémoire interne qui leur permet de traiter des séquences. Appliquer un dropout classique d’une couche à l’autre pourrait perturber cette mémoire et rendre l’apprentissage instable. Le *Variational Dropout* propose une approche où les mêmes neurones sont désactivés à chaque pas de temps au lieu d'être réinitialisés à chaque itération, permettant une meilleure stabilité tout en préservant la capacité d’apprentissage de longues dépendances.

Les architectures basées sur les *Transformers*, telles que *BERT* et *GPT*, utilisent également le dropout pour réduire le surapprentissage. Dans ces modèles, le dropout est appliqué après les couches *self-attention* et *feed-forward*, ce qui empêche les ténors du modèle de trop se fier à certaines unités. Cette approche est essentielle pour garantir que le modèle généralise bien aux contextes qu’il n’a pas rencontrés pendant l’entraînement.

### 4.2 Combinaison du Dropout avec d’autres techniques

Le dropout peut être encore plus efficace lorsqu’il est combiné à d’autres méthodes de régularisation. Lorsqu'il est couplé avec la *Batch Normalization*, les activations sont normalisées avant d’appliquer le dropout, ce qui stabilise l’apprentissage et accélère la convergence. Toutefois, l’ordre d’application est critique : appliquer le dropout avant une normalisation peut perturber la distribution des activations et réduire l’effet bénéfique de la normalisation.

Une autre combinaison efficace est celle du dropout avec la régularisation *L2* (*Weight Decay*). Alors que le dropout empêche la co-adaptation des neurones en masquant certaines activations, la régularisation L2 impose une contrainte directe sur la magnitude des poids, évitant ainsi des poids excessivement grands. En combinant ces deux méthodes, on obtient un réseau plus stable et mieux généralisé.

Des techniques plus avancées ont également été développées à partir du dropout, telles que le *DropConnect*, qui ne désactive pas les neurones mais certains poids du réseau, le *ZoneOut*, qui permet de conserver des activations inchangées dans les RNNs, et le *Monte Carlo Dropout*, qui applique le dropout aussi durant l’inférence pour estimer la variabilité des prédictions et produire une estimation de l’incertitude.

### 4.3 Limites et Précautions d’utilisation du Dropout

Bien que puissant, le dropout ne fonctionne pas toujours de manière optimale. Un taux de dropout trop élevé peut entraîner une perte excessive d’informations et une sous-apprentissage du modèle, tandis qu’un taux trop faible ne produit aucun effet notable. Une calibration minutieuse est donc essentielle.

L’impact sur la convergence est également un élément à considérer. Le dropout ajoute du bruit au processus d’apprentissage, ce qui ralentit la convergence, notamment dans les réseaux profonds. L’utilisation de techniques comme l’ajustement dynamique du *learning rate* ou l’adoption d’optimiseurs adaptatifs tels qu’Adam permet de compenser cet effet.

Enfin, dans certains cas, le dropout n’est pas la meilleure stratégie de régularisation. Lorsque les données d’entraînement sont limitées, des méthodes comme la *data augmentation* peuvent s’avérer plus efficaces. De même, certaines architectures modernes comme les *ResNets* bénéficient davantage des connexions résiduelles que du dropout pour assurer une meilleure généralisation.

***
## Conclusion
Le dropout est une méthode de régularisation très efficace qui peut être généralisée à divers types de réseaux neuronaux et combinée avec d’autres techniques pour améliorer la généralisation. Le dropout est assez facile à implémenter car Pytorch propose des fonctions très intuitives pour appliquer le dropout. Cependant, il doit être ajusté correctement pour éviter une perte excessive d’information ou un ralentissement de l’apprentissage. Il est donc essentiel de tester différentes configurations et de l’adapter au contexte du modèle utilisé.

***
## Références
1. <a href="references/Starting paper.pdf" target="_blank"> *Dropout: A simple way to prevent neural networks from overfitting*</a> (2014). Srivastava, N., Hinton, G., Krizhevsky, A., Sutskever, I., & Salakhutdinov, R.
2. <a href="references/Starting paper.pdf" target="_blank"> *Efficient Object Localization Using Convolutional Networks*</a> (2014). Jonathan Tompson, Ross Goroshin, Arjun Jain, Yann LeCun, Christoph Bregler, New York University. 
3. <a href="https://openai.com/index/chatgpt/" target="_blank">🐱 Mon ami le chat</a>

